<a href="https://colab.research.google.com/github/Samiha-H/CODSOFT/blob/main/credit_card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'default-of-credit-card-clients-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F306%2F666%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240707%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240707T123201Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D70e89532148edee0f4faa6ba66dffd9c72542164cc14969830b006f8dd9dc8601c79fa4ffca7795b1d7bb5ca13f2adb80ec8e91076c7cd17aa9135ec2646e538cf14123c7065b157d6b7f326a9f91e6d3064b89a51f1ac66a1ef8112c274697087fe11538528d65934a8a5bbbd0abd6efad9efc1c764b9b06a3c8b16f48f17dc55b5a69afc3127c6607a84605f3f5b84ddfae7f81b867c026db72e2b16a3206dda416452a5e6901a3075bde0cc67a59e2ff9d6699190cea4a60631b44525d3918043764db5be8a59b6ee0e778e57dff396ee6cfaf9b350a9f2dc8ed41d6dac103c8ce38146379928631df7420a8b3285c7a278129ce1e139d121fc5df386f802,german-credit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F531%2F1056%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240707%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240707T123201Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D97d6c8d3cb7908e0b5d66a22a8559773ba2ec694c4f4adb67b2563a621a5bd18bb013dd0919309d77d5440414843092bb0a1adb2519d94904dfda35a479f3d79840fab66bea63d18de03096c61ad76f930ea514c7c9cc36a305143daf98a41df6dd1ff5d35e2bcef07afe8c3b4e9c713a5d260a9fbea893113dcbf02f05f433b6813e875b8fa7e804dcf65e41211cba9a8faf20c15fce1497b970bbc24f1592b425c08d5821e920297b4aa8ccffc0af1d81487d05654169ba96f6ec04fd8df336ee18701b43eb22c98a95fdf87393ce5a2c04958ef195411addca9611195d01e85c5643d284eb9cd63e0d171d3d1d43e2e83d1ac144127216b71e286863c733c,creditcardfraud:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F310%2F23498%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240707%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240707T123201Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D00ee556c5165c747564704690200a8ff81a72cd9bbb815b77059676689f40602896a697dbd870d512a63cd119d5e992a63c569252d0f625e235c1e7232fe5206d23b64e614883ebff67c8a5ef4f7dedc93c2d0c7ba6b5f3569d95f501c0fd96a7d9f0e5ffac5ca746ed9001cd2a57619ab03c871534b70943e202d2379b4d6c746753642e58392545ce7ecc12ff9dbf72fe6d53998c208885f5052c1c81cdf7b0b1187af6b50d942000d4f6e1038333a8d9246d56d825661a73cbe98b9b13a1f229f1cdeb48570d1692a28de044b5e7d13a20393bbf744cfb3dd9297c3307c82f8397068562751b74fc51dbee4be1e5fe3c7f78fba48c569536b9dbb6840f7f6,credit-card:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F263888%2F552038%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240707%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240707T123201Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D40b8cb2f0867f19daa792189d78117039109a8ec0c1a20eaf512f9829789a7568347528a7b18cc02ee48cd326e7e181dc57b91bc6f452d82ad66cad62b31bbff5617ed977720a9573acda15903f96249ac79afcbbef0444ee25796cb85286d0e02f6c10206bf1a2ae4024bdb50fb57463b734b63439d007f896abddf88783c4c5a93b70a5beda9f2bd804ce208e5c45deb79cada4ee47535bf0d40070624a81730c4a810e60c7e2f3d1414a31e144a7b04b6ab7b342e9f303c7dbe11b7ab25d536ff828986367af586a26d4da2a285fa07135b319e6cbbda26595d4055bac46f256dd20ac5a61fc66e71960765bf613e25433413c22bbf4de38eec582a5f7459,fraud-detection:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F817870%2F1399887%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240707%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240707T123201Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8d8417af7d89407df5a451955cb4e86333b70f3a0372604b43a7c67dd83e9ee84ef6d600a5fdd95ad7ab25a6ffad225cb193a09065bb37419b126b83bf7a8032359301e8cef03c6f127b8d9d4328823fa5e00443863c7c536b04eab890b07a7c82cc18910d9cfbf22d1c52302dd4694a49e1f1b2abba61142e2180b13689df771959667af550c04e60989bacca3abd2fc7cf2a455ff45d483a1ea592107af05b6bde2ecc15891504ca0fb4d3e0965b448b8c93cd10cb5f8ca90966c0f71a08e37fbabcc2d71246f2836d0eede0d3a6c59e65fe3f179ce9958c3f56049206152e30ed66650e4560bfde55723a8461f3af60f1b11745c29aeaf0b3be3f5ff0d3c9,credit-card-customers:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F982921%2F1660340%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240707%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240707T123201Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7bd5010a0d3d5820e605c6b941a09c63f7fd7c09a53e67674b4197dbe26a3bbc032041cc20c1b884cc2b5bd32963bbc2612005fe49c2c2d943d3778bcd38457d901c9b59ccc86e0d8df946ea6157333bb6a8cbcc3a49c4ceb85933229ea3eb13b9fec08f66aec63a981f3f3cae401309f9d9415c59b080ab80453d929ec4bfdc7e4f29b9a5b840ed0b5ed02b5ae242c8949063c94ca2b7e7f00207960b2d1f400f32c1c22be6ddef650e00cfc9225817b6f09e3581d447c201a852f29e462008d1d5e440e39c13a34dbc9acdf767d9d4058aac05995bc7da77938ba1244fc7bb13a447dc6152dce07b9d8ba69f6ea86b374bee571b2072dba7fbdabeec0e8ac5,credit-card-customers-prediction:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2590623%2F4422779%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240707%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240707T123201Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D396a349584da765718d51745c6c2be711f40f6e716a40eec60abcadde18381adb4826d5c53e4cdc2e16e128537e89430ac4aaf9dc4e50fb603684f1bf4e0a857185ef4a9277fe14868151ef3d65f5590e116348cc9f3f24165b29c245d60cd5652d572d11aa937e65a509b078feb70acb95774a79f4d4d21dd699b8a1b5d79a8bdf8c747251995afbf13298d9d0b139fa13d4ed99e0af0ec03d4fe40814bc3850a5915f9a04b4cfa1e982878d974c06d9cf0b9484733d27da7b705ed3550f6e34c4a9f1b78155f2124cca95bedf9c4d6a59a6604ca2539cd9e38ebea517634d20612c972db86da8a9ce71e7f700d3422862195f701420ced2872948ec9a6af54,credit-card-spendings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4541632%2F8050363%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240707%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240707T123201Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4155112cb4b39b146281a3e8ce080e1039b6c954a1b59d422fca0b87b0c211df546991da049cc00f19af729268df5047acdd6899031698af12df3dd775e1ab3ff9922259c37554c03c44d1098eedb8b66b5332af930b6b984b36938f39726b8257026e2ffa032ea07b4a90ec1bf7456d1bee403985b7c35bfaa111bdb39489287a0398315ee915fbb879653752e16a61d12d93ff0e7f76625b4461d582270d66d9be8291da2771f63cd2206fe723e05a63c97aeb9a8db49905a834de055d2021236f994cfad717d3340dad2b40406835aee0bc8b389ae261add0c2633d3c6ea71dde72ed921eecccf992f0ab184a9589814fd1ce6e864065f846a81407c7cdce'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd

In [ ]:
data =pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

In [ ]:
data.head()

In [ ]:
pd.options.display.max_columns = None

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.shape

In [ ]:
print("Number of columns: {}".format(data.shape[1]))
print("Number of rows: {}".format(data.shape[0]))

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()
data['Amount'] = sc.fit_transform(pd.DataFrame(data['Amount']))

In [ ]:
data.head()

In [ ]:
data = data.drop(['Time'], axis =1)

In [ ]:
data.head()

In [ ]:
data.duplicated().any()

In [ ]:
data = data.drop_duplicates()

In [ ]:
data.shape

In [ ]:
data['Class'].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
sns.countplot(data['Class'])
plt.show()

In [ ]:
X = data.drop('Class', axis = 1)
y=data['Class']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

classifier = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree Classifier": DecisionTreeClassifier(),
    "Random Forest Classifier": RandomForestClassifier(),
    "Support Vector Classifier": SVC(),
    "K-Nearest Neighbors Classifier": KNeighborsClassifier(),
    "Gaussian Naive Bayes": GaussianNB(),
    "AdaBoost Classifier": AdaBoostClassifier(),
    "Gradient Boosting Classifier": GradientBoostingClassifier(),
    "Bagging Classifier": BaggingClassifier(),
    "Extra Trees Classifier": ExtraTreesClassifier(),
    "Stochastic Gradient Descent Classifier": SGDClassifier(),
    "Voting Classifier": VotingClassifier(estimators=[
        ('lr', LogisticRegression()),
        ('dt', DecisionTreeClassifier()),
        ('rf', RandomForestClassifier()),
        ('svc', SVC()),
        ('knn', KNeighborsClassifier())
    ], voting='hard')
}

for name, clf in classifier.items():
    print(f"\n=========={name}===========")
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Evaluation Metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"\n Accuracy: {accuracy}")
    print(f" Precision: {precision}")
    print(f" Recall: {recall}")
    print(f" F1 Score: {f1}")

    # Confusion Matrix
    print("\n Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    # Classification Report
    print("\n Classification Report:")
    print(classification_report(y_test, y_pred))


In [ ]:
# Undersampling

In [ ]:
normal = data[data['Class']==0]
fraud = data[data['Class']==1]

In [ ]:
normal.shape

In [ ]:
fraud.shape

In [ ]:
normal_sample = normal.sample(n=473)

In [ ]:
normal_sample.shape

In [ ]:
new_data = pd.concat([normal_sample,fraud], ignore_index=True)

In [ ]:
new_data.head()

In [ ]:
new_data['Class'].value_counts()

In [ ]:
X = new_data.drop('Class', axis = 1)
y= new_data['Class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
classifier = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree Classifier": DecisionTreeClassifier()
}

for name, clf in classifier.items():
    print(f"\n=========={name}===========")
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"\n Accuaracy: {accuracy_score(y_test, y_pred)}")
    print(f"\n Precision: {precision_score(y_test, y_pred)}")
    print(f"\n Recall: {recall_score(y_test, y_pred)}")
    print(f"\n F1 Score: {f1_score(y_test, y_pred)}")

In [ ]:
# OVERSAMPLING

In [ ]:
X = data.drop('Class', axis = 1)
y= data['Class']

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
X_res, y_res = SMOTE().fit_resample(X,y)

In [ ]:
y_res.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size = 0.2, random_state = 42)

In [ ]:
classifier = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree Classifier": DecisionTreeClassifier()
}

for name, clf in classifier.items():
    print(f"\n=========={name}===========")
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"\n Accuaracy: {accuracy_score(y_test, y_pred)}")
    print(f"\n Precision: {precision_score(y_test, y_pred)}")
    print(f"\n Recall: {recall_score(y_test, y_pred)}")
    print(f"\n F1 Score: {f1_score(y_test, y_pred)}")

In [ ]:
dtc = DecisionTreeClassifier()
dtc.fit(X_res, y_res)

In [ ]:
import joblib

In [ ]:
joblib.dump(dtc, "credit_card_model.pkl")

In [ ]:
model = joblib.load("credit_card_model.pkl")

In [ ]:
pred = model.predict([[-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942518,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,0.0907941719789316,-0.551599533260813,-0.617800855762348,-0.991389847235408,-0.311169353699879,1.46817697209427,-0.470400525259478,0.207971241929242,0.0257905801985591,0.403992960255733,0.251412098239705,-0.018306777944153,0.277837575558899,-0.110473910188767,0.0669280749146731,0.128539358273528,-0.189114843888824,0.133558376740387,-0.0210530534538215,149.62]])

In [ ]:
pred[0]

In [ ]:
if pred[0] == 0:
    print("Normal Transcation")
else:
    print("Fraud Transcation")